In [1]:
! pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import random
import json
import pandas as pd

In [2]:
search_name = "Духи"
articul_num = "1713520"
post_place = "г Москва, Улица Лазо 4к1"

In [74]:
_bot = bot
bot = Bot(driver=_bot.driver)

In [15]:
# Python Modules
import os
import json
import logging
import pandas as pd

# For telegram api
# pip install python-telegram-bot --upgrade
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext, \
    CallbackQueryHandler, TypeHandler, ConversationHandler



In [4]:
%run Bot.py

file_path = "users_data/794329884_order.xlsx"
inside=True
df = pd.read_excel(file_path)

orders = [row.tolist() for i, row in df.iterrows()]

max_bots = max([order[3] for order in orders])
bots = [Bot(name=i) for i in range(1)]
_bots = bots[:max_bots]
data_for_bots = [[] for _ in range(max_bots)]

for _, order in enumerate(orders):
    if inside:
        article, search_key, quantity, pvz_cnt, _, __ = order
    else:
        article, search_key, quantity, pvz_cnt = order

    for i in range(max_bots):
        if pvz_cnt > 0:
            data_for_bots[i] += [[article, search_key, quantity]]
        pvz_cnt -= 1
print(data_for_bots)

post_places = [order[5] for order in orders][:max_bots]
order_id = str(orders[0][4])

print(order_id, post_places)

report = []
for i, bot in enumerate(_bots):
    report += bot.buy(data_for_bots[i], post_places[i], order_id)

[[[44398622, 'Серьги для девочек', 1]]]
123 ['г. Москва, м Семёновская, Измайловское шоссе, д. 6']
c17273059 ['44398622']
c72635552 ['44398622']
c12470742 ['44398622']
c31353680 ['44398622']
c16678199 ['44398622']
c50215375 ['44398622']
c15805127 ['44398622']
c19278335 ['44398622']
c11036625 ['44398622']
c41917843 ['44398622']
c60304982 ['44398622']
c70095670 ['44398622']
c35951389 ['44398622']
c70414166 ['44398622']
c48992787 ['44398622']
c48989547 ['44398622']
c30315850 ['44398622']
c39312948 ['44398622']
c25790014 ['44398622']
c44398622 ['44398622']
['44398622']
['44398622']


TypeError: can only concatenate str (not "int") to str

In [16]:
def choose_post_place(adress):
    bot.driver.find_element(By.XPATH, '//h2[text()="Способ доставки"]/../../div[text()="Выбрать адрес доставки"]').click()
    sleep(1)
    bot.driver.find_element(By.XPATH, '//button[text()="Выбрать адрес доставки"]').click()
    sleep(2)
    bot.driver.find_element(By.XPATH, '//input[@placeholder="Введите адрес"]').send_keys(adress)
    sleep(2)
    bot.driver.find_element(By.XPATH, '//ymaps[text()="Найти"]').click()
    sleep(2)
    try:
        bot.driver.find_element(By.XPATH, '//ymaps[contains(@class, "__first")]').click()
    except:
        pass
    sleep(2)
    bot.driver.find_element(By.XPATH, '//span[contains(text(), "'+ adress +'")]').click()
    sleep(2)
    bot.driver.find_element(By.XPATH, '//div[@class="balloon-content-block"]/button').click()
    sleep(2)
    bot.driver.find_element(By.XPATH, '//button[@class="popup__btn-main"]').click()
    sleep(2)

In [17]:
choose_post_place(post_places[0])

In [13]:
import random

In [20]:
self = bot
number = '9104499982'
def get_qr_code(file_name):
    try:
        self.driver.find_element(By.XPATH, '//button[text()="                Оплатить заказ                "]').click()
        sleep(2)
        self.driver.find_element(By.XPATH, '//button[contains(@class,"popup__btn-main")]').click()
        sleep(2)
        svg = self.driver.find_element(By.XPATH, '//div[@class="qr-code__value"]')
        self.save_qr_code(svg, file_name)
        sleep(2)
        self.driver.find_element(By.CLASS_NAME, 'popup__close').click()
    except:
#         Utils.login(number)
        login(self.driver, number)
        input("Нажми Enter, зайка")
        get_qr_code(file_name)

In [21]:
get_qr_code('file_name.svg')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".j-main-login"}
  (Session info: chrome=100.0.4896.75)
Stacktrace:
Backtrace:
	Ordinal0 [0x01039943+2595139]
	Ordinal0 [0x00FCC9F1+2148849]
	Ordinal0 [0x00EC4528+1066280]
	Ordinal0 [0x00EF0FD4+1249236]
	Ordinal0 [0x00EF11CB+1249739]
	Ordinal0 [0x00F1D812+1431570]
	Ordinal0 [0x00F0BA34+1358388]
	Ordinal0 [0x00F1BAF2+1424114]
	Ordinal0 [0x00F0B806+1357830]
	Ordinal0 [0x00EE6086+1204358]
	Ordinal0 [0x00EE6F96+1208214]
	GetHandleVerifier [0x011DB232+1658114]
	GetHandleVerifier [0x0129312C+2411516]
	GetHandleVerifier [0x010CF261+560433]
	GetHandleVerifier [0x010CE366+556598]
	Ordinal0 [0x00FD286B+2173035]
	Ordinal0 [0x00FD75F8+2192888]
	Ordinal0 [0x00FD76E5+2193125]
	Ordinal0 [0x00FE11FC+2232828]
	BaseThreadInitThunk [0x763BFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77787A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77787A4E+238]


In [8]:
def login(driver, number):
        # vars # 1
        to_login_btn = driver.find_element(By.CLASS_NAME, 'j-main-login')

        # resolve # 1
        to_login_btn.click()
        sleep(random.uniform(3, 5))
        
        # vars # 2
        login_input = driver.find_element(By.CLASS_NAME, 'input-item')
        login_btn = driver.find_element(By.CLASS_NAME, 'login__btn')

        # resolve # 2
        login_input.send_keys(number)
        sleep(random.uniform(2, 4))
        login_btn.click()

In [23]:
[]+[{'a': '2'}]

[{'a': '2'}]

In [8]:
adresses = ['123', '321']

In [13]:
"".join(map(lambda x: x+'\n', adresses))[:-1]

'123\n321'

In [34]:
d

""
